## Imports

In [38]:
import numpy as np
import tiktoken  # Tokenizer-Bibliothek für GPT-2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 1. Tokenisierung:
- Tokenisierung ist der Prozess der Umwandlung von Text in kleinere Einheiten (Tokens).
- Tokens können Zeichen ("a", "b"), Wörter ("Hallo", "Baum") oder Subwörter ("Ha", "Ba") sein.
- LLMs arbeiten mit Token-Sequenzen anstelle von Rohtext → Tokenisierung ist daher das Vorbereiten des Textes für die Eingabe in das LLM :)




#### 1.1 Datensatz erstellen:

In [39]:
class DatasetForGPT(Dataset):
    def __init__(self, max_length, stride, tokenizer, txt):
        """
        Erstellt ein Dataset aus einem Text für ein GPT-basiertes Modell.

        Parameter:
        - txt: Der Eingabetext als String. (z.B. ein Opensource Buch der Seite Projekt Gutenberg https://www.projekt-gutenberg.org/)
        - tokenizer: Der GPT2-Tokenizer zur Tokenisierung des Textes.
        - max_length: Maximale Länge einer Token-Sequenz.
        - stride: Schrittweite für die Erstellung überlappender Sequenzen.
        """
        self.input_ids = []  # Liste zur Speicherung der Eingabe-Tokens
        self.target_ids = []  # Liste zur Speicherung der Ziel-Tokens (verschobene Sequenz)

        # Den gesamten Text in Tokens umwandeln
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Erzeuge überlappende Sequenzen aus den Token-IDs (siehe Erklärung 1)
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]  # Eingabesequenz
            target_chunk = token_ids[i + 1: i + max_length + 1]  # Zielsequenz (verschoben um 1 Token -> siehe Bild 1: Erklärung 1 )

            # Speichern der Tensor-Repräsentation der Sequenzen
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        """ Gibt die Anzahl der Trainingsbeispiele zurück."""
        return len(self.input_ids)

    def __getitem__(self, idx):
        """ Gibt ein Trainingsbeispiel bestehend aus (Eingabe, Ziel) zurück."""
        return self.input_ids[idx], self.target_ids[idx]


##### Erklärung 1
Das bedeutet:
- Das Modell bekommt eine Eingabesequenz (input_chunk), z. B. "LLMs learn to predict one"
- Die Zielsequenz (target_chunk) ist dann dieselbe Sequenz, aber um ein Token nach rechts verschoben, sodass das Modell lernen soll, das nächste Token vorherzusagen.

→ Genau wie im Bild (Quelle: Raschka 2025)
###### Bild 1 Sliding Window Approach
![Image 2 Sliding Window Approach](images/Image1_sliding_window_approach.png)

Das Modell sieht nur den bisherigen Kontext (blau markiert).
Das Modell soll das nächste Wort (rot markiert) vorhersagen.
Es kann zukünftige Wörter nicht direkt sehen, sondern muss sie aus den bisherigen Token ableiten. Während des Trainings bekommt das Modell eine Sequenz (input_chunk) als Eingabe und versucht, das nächste Token (target_chunk) vorherzusagen. Durch viele Wiederholungen lernt das Modell dann grammatische Strukturen, Satzbedeutungen und sogar komplexe Zusammenhänge.

#### 1.2 Dataloader erzeugen:

Der DataLoader hilft, die Daten effizient für das LLM bereitzustelle.


In [40]:
def create_dataloader(txt, batch_size=4, max_length=256,
                      stride=128, shuffle=True, drop_last=True, num_workers=0):
    """
    Erstellt einen DataLoader für das Training eines LLMs.

    Parameter:
    - txt: Eingabetext als String.
    - batch_size: Anzahl der Samples pro Batch.
    - max_length: Maximale Token-Sequenzlänge.
    - stride: Schrittweite für die Erzeugung überlappender Sequenzen.
    - shuffle: Ob die Reihenfolge der Sequenzen zufällig gemischt wird.
    - drop_last: Ob das letzte Batch verworfen wird, falls es unvollständig ist.
    - num_workers: Anzahl der Threads für die Datenverarbeitung.

    Rückgabe:
    - Ein DataLoader-Objekt für das Training.
    """
    # Initialisiere den GPT-2 Tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Erstelle das Dataset
    dataset = DatasetForGPT(max_length, stride, tokenizer, txt)

    # Erstelle den DataLoader aud der torch Lib (Erklärung 2):
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
    #Shuffle sorgt für zufällige Reihenfolge -> Damit das Modell nicht immer dieselbe Reihenfolge der Daten sieht (wichtig für das Training).
    # Drop_last vermeidet ungleich große Batches
    # Falls die Anzahl der Daten nicht genau durch batch_size teilbar ist, werden unvollständige Batches verworfen.

    return dataloader


##### Erklärung 2:

Ein DataLoader bereitet die Daten für das Modelltraining vor. In diesem Fall fasst der Data Loader mehrere Sequenzen (des Sliding Window Approaches) zu Batches zusammen:

Das Modell verarbeitet nicht einen einzelnen Satz nach dem anderen, sondern mehrere Sequenzen gleichzeitig (z. B. 4 auf einmal, wenn batch_size=4 ist).
###### Beispiel:
**Batch 1:**
- Eingabe:  `["LLMs learn to predict", "learn to predict one", "to predict one word", "predict one word at"]`
- Ziel:     `["learn to predict one", "to predict one word", "predict one word at", "one word at a"]`

**Batch 2:**
- Eingabe:  `["one word at a", "word at a time", "at a time <|endoftext|>", "..."]`
- Ziel:     `["word at a time", "at a time <|endoftext|>", "...", "..."]`




#### 1.3 Code testen

In [41]:
with open("test_text", "r") as f:
    test_text = f.read()

# DataLoader erstellen
dataloader = create_dataloader(test_text, batch_size=2, max_length=6, stride=3)

# Ersten Batch ausgeben
for batch in dataloader:
    inputs, targets = batch
    print("\n=== Erster Batch ===")
    print("Input Shape:", inputs.shape)   # Erwartet: (batch_size, max_length)
    print("Target Shape:", targets.shape) # Erwartet: (batch_size, max_length)
    print("\nEingabe Batch:", inputs)
    print("Ziel Batch:", targets)
    break  # Nur den ersten Batch ausgeben




=== Erster Batch ===
Input Shape: torch.Size([2, 6])
Target Shape: torch.Size([2, 6])

Eingabe Batch: tensor([[  257,  2726,  6227,   284,  1833,   683],
        [ 1534,  4241,   286, 19217,   290, 18876]])
Ziel Batch: tensor([[ 2726,  6227,   284,  1833,   683,  1365],
        [ 4241,   286, 19217,   290, 18876,  5563]])


#### 1.4 Zusammenfassung Tokenisierung
- Das DatasetForGPT erzeugt nur die überlappenden Sequenzen (Sliding Window Approach, siehe Bild 1) und speichert bereits alle Input- und Target Sequenzen
- Der DataLoader fasst diese dann in Batches zusammen, die dann im Training verwendet werden können.

**Multi-Head Attention aus Kapitel 2**

In [43]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec

# 4. Implemetierung des GPT-Modells:
- Konfiguration des GPT-Modells.
- Die Architektur ist wie folgt aufgebaut:
    - Grundgerüst der Architektur, Layer normalization, GELU-Aktivierung, Feed forward network, Shortcut connections, Transformer block.
- Transformer blocks sind eine zentrale strukturelle Komponente von GPT-Modellen - Diese kombinieren masked multi-head attentions Module mit vollständig verbundenen feed forward networks, die die GELU activation function nutzen
###### Bild 3 Visualisierung der Architektur
![Image 3 Visualisierung der Architektur](images/LLM-Architektur.png)

#### 4.1 Eine LLM-Architektur programmieren

**Config des GPT-Modells**

In [44]:
GPT_CONFIG_124M = {
"vocab_size": 50257, # Vokabular an Wörtern - Byte Pair Encoding (BPE)-Tokenizer
"context_length": 1024, # Maximale Anzahl an Input-Tokens
"emb_dim": 768, # Embedding size - transformiert jedes Token in ein 768-dimensionalen Vektor
"n_heads": 12, # Anzahl der attention heads
"n_layers": 12, # Anzahl der Transformer-Blöcke
"drop_rate": 0.1, # Dropout Mechanismus - Overfitting vermeiden durch dropouts (0.1 sind somit 10% zufülliger dropout)
"qkv_bias": False # Bestimmt ob ein Bias-Vektor in der linearen Layer der Multi-head-atention beinhaltet sein soll
}

Zur Implementierung wird ein batch tokenisiert der aus zwei Text-Inputs für das GPT-Modell besteht.

! *Es wird hierfür der Tiktoken-Tokenizer aus Kapitel 2 verwendet* !

##### 4.2 Normalizing Activation mit Layer normalization
- Hauptidee der Layer normalization besteht in der Anpassung der Activations (Outputs) einer neural network layer. Es soll somit einen Mittelwert von 0 und eine Varianz von 1 haben (Unit Variance)
- Beschleunigt die Konvergenz zu effektiven Gewichten und gewährleistet ein konistentes und zuverlässiges Training.
- Die Layer normalization wird in modernen Transformer-Architekturen in der Regel vor und nach dem Multi-Head-Attention-Modul angewendet.


In [45]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim): #emb_dim = embedding dimension (letze Dimension des input tensors x / Größe der Vektoren, die die Token im Modell repräsentieren)
        super().__init__()
        self.eps = 1e-5          # Dadurch wird vermieden, dass es zu einer Division durch 0 kommt, falls die Varianz extrem klein oder genau 0 ist.

        # Nach der Normalisierung wären alle Werte auf eine Standardnormalverteilung mit Mittelwert = 0 und Varianz = 1 gebracht
        # Damit das Modell aber flexibel bleibt, gibt es zwei trainierbare Parameter:
        self.scale = nn.Parameter(torch.ones(emb_dim)) # Multiplikationsfaktor, der die Werte nach der Normalisierung wieder streckt oder komprimiert.
        self.shift = nn.Parameter(torch.zeros(emb_dim)) # Ein Wert, der hinzugefügt wird, um die Normalisierung nach oben oder unten zu verschieben.

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False) # 
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

**Testen und Darstellung der Layer normalization anhand eines Beispiels**

In [21]:
# Erstellung von 2 trainings-beispielen mit fünf Dimensionen
torch.manual_seed(123)
batch_example = torch.randn(2, 5)
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
out = layer(batch_example)
print("Erstellung von 2 trainings-beispielen mit fünf Dimensionen: \n")
print(out, "\n")

# Mittelwert und Varianz ohne Layer normalization:
mean = out.mean(dim=-1, keepdim=True)
var = out.var(dim=-1, keepdim=True)
print("Mittelwert und Varianz ohne Layer normalization: \n")
print("Mean:\n", mean)
print("Variance:\n", var, "\n")


# Layer normalization mit batch-input
ln = LayerNorm(emb_dim=5)
out_ln = ln(batch_example)
mean = out_ln.mean(dim=-1, keepdim=True)
var = out_ln.var(dim=-1, unbiased=False, keepdim=True)
print("Layer normalization mit batch-input: \n")
print("Mean:\n", mean)
print("Variance:\n", var)

Erstellung von 2 trainings-beispielen mit fünf Dimensionen: 

tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>) 

Mittelwert und Varianz ohne Layer normalization: 

Mean:
 tensor([[0.1324],
        [0.2170]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[0.0231],
        [0.0398]], grad_fn=<VarBackward0>) 

Layer normalization mit batch-input: 

Mean:
 tensor([[-2.9802e-08],
        [ 0.0000e+00]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


##### 4.3 Implementierung eines feed forward network mit GELU activations
- GELU (Gaussian error linear unit)
- Kleines neuronales Netzwerk Submodule welches als Teil des transformer blocks in LLMs genutzt wird.
- Aktivierungsfunktionen, die Gaußsche bzw. sigmoidale lineare Einheiten enthalten und bieten eine verbesserte Leistung für Deep-Learning-Modelle.

*Funktion kann als PyTorch module implementiert werden*

In [46]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

# Wird später im Transformer Block verwendet
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)
    
# FeedForward module mit einer Token embedding size von 768 und feed in einen batch input mit zwei samples und jeweils drei Token     
ffn = FeedForward(GPT_CONFIG_124M)
x = torch.rand(2, 3, 768)
out = ffn(x)
print("Output Tensor ist der selbe wie im input Tensor: ")
print(out.shape)

Output Tensor ist der selbe wie im input Tensor: 
torch.Size([2, 3, 768])


##### 4.4 Shortcut Connections
- Werden auch skip or residual connections genannt.
- Verminderung der Herausforderung des vanishing gradients (Gradienten werden immer kleiner was es schwierig macht, frühere Layers effektiv zu trainieren).
- Gradienten "leiten" die updates der Gewichte während des Trainings.  

*Ermöglichen einen alternativen kürzeren Weg für den Gradienten der durch das Netz fließt indem eine oder mehrere Schichten übersprungen werden indem der Output einer Layer zum Oustput einer späteren layer hinzugefügt wird - Wichtige Rolle während des backward pass im Training*

###### Bild 4 Visualisierung Shortcut Connection
![Image 4 Visualisierung Shortcut Connection](images/Shortcut-Connection.png)

##### 4.5 Verbinden von Attention und Linearen Layers in einem Transformer-Block
- Der Transformer-Block kombiniert die bereits thematisierten Konzepte wie Multi-Head-Attemtion, layer normalization, dropout, feed forward layers und GELU activations.
- Die Operationen innerhalb des Transformer-Blocks, einschließlich der Multi-Head-Attention- und Feed-Forward-Layers sind darauf ausgelegt, die Vektoren so zu transformieren, dass ihre Dimensionalität erhalten bleibt.
- Die Idee ist, dass der Self-Attention-Mechanism im Multi-Head-Attention-Block Beziehungen zwischen Elementen in der Input-Sequenz identifiziert und analysiert

In [23]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut Connection für den attention-block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)   
        x = self.drop_shortcut(x)
        x = x + shortcut # Den orginalen input hinzufügen

        # Shortcut connection für den feed-forward-block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut # Den orginalen input hinzufügen

        return x


**Testen und Darstellung des Transformer-Blocks anhand eines Beispiels**

In [24]:
# Transformer-Block mit Beispieldaten
# Dieser behält Input Dimensionen im Output und zeigt, dass die Transformer-Architektur 
# Datenfolgen verarbeitet, ohne ihre Form im gesamten Netzwerk zu verändern.
torch.manual_seed(123)
x = torch.rand(2, 4, 768)
block = TransformerBlock(GPT_CONFIG_124M)
output = block(x)
print("Input shape:", x.shape)
print("Output shape:", output.shape, "\n")

Input shape: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768]) 



##### 4.6 Das GPT-Modell coden
Zusammenführung der zuvor thematisierten Punkte

###### Bild 5 Visualisierung Datenfluss der GPT-Modell-Architektur
![Image 5 Visualisierung Datenfluss der GPT-Modell-Architektur](images/GPT-Modell-Architektur.png)

In [25]:
class GPTModel(nn.Module):
    def __init__(self, cfg): # init constructor initialisiert die Token- und positional embedding Layers durch die Übergabe der Konfiigurationen aus des Python dictionary (cfg)
        super().__init__()  # Embeddings sind für die Umwandlung der eingegebenen Token-Indizes in dichte Vektoren und das Hinzufügen von Positionsinformationen verantwortlich
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # Erstellung sequenzeller Stapel (stack) von Transformer-Blöcken. Gleiche Anzahl wie die definierten Layers in cfg.
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        # LayerNorm (Outputs der Transformer-Blöcke standardisieren, um den Lernprozess zu stabilisieren)
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    # Nimmt einen batch an input-token-indizes, berechnet deren embeddings, wendet positional embeddings an, 
    # leitet die Sequenz durch die Transformer-Blöcke, normalisiert den Output und berechnet dann die Logits, 
    # die die nicht normalisierten Wahrscheinlichkeiten des nächsten Tokens darstellen
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

**weight tying**
- 163M statt 124M Parameter
- Wurde in der originalen GPT-2 Architektur genutzt - Wiederverwendung die Gewichte der Token embedding Layer in der Output Layer

In [32]:
model = GPTModel(GPT_CONFIG_124M)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

total_params_gpt2 = (
total_params - sum(p.numel()
for p in model.out_head.parameters())
)
print(f"Number of trainable parameters "
f"considering weight tying: {total_params_gpt2:,}"
)

total_size_bytes = total_params * 4
total_size_mb = total_size_bytes / (1024 * 1024)
print(f"Total size of the model: {total_size_mb:.2f} MB")

Total number of parameters: 163,009,536
Number of trainable parameters considering weight tying: 124,412,160
Total size of the model: 621.83 MB


##### 4.7 Text generieren
- Implementierung einer generativen Schleife für ein Language Model mit PyTorch
- Durchläuft eine bestimmte Anzahl neu zu erzeugender Token und berechnet Vorhersagen und wählt dann das nächste Token auf der Grundlage der Vorhersage mit der höchsten Wahrscheinlichkeit aus.

In [47]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx (Batch & Tokens) ist ein Array von Indizes im aktuellen context 
    for _ in range(max_new_tokens):

        # Cropt den aktuellen Kontext wenn das LLM bspw. nur 5 tokens unterstützt, der Context aber 10 ist, werden nur die letzten 5 Tokens als Context genutzt
        idx_cond = idx[:, -context_size:]

        # Vorhersagen bekommen
        with torch.no_grad():
            logits = model(idx_cond)

        # Fokussiert sich auf den letzten Schritt
        # (batch, n_token, vocab_size) wird zu (batch, vocab_size)
        logits = logits[:, -1, :]

        # Idx mit dem höchsten logits wert
        idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch, 1)

        # Anhängen des sampled index an die laufende Sequenz
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx


**Testen und Darstellung der Textgeneration anhand eines Beispiels**

*Modell kann noch keinen kohärenten Text produzieren, da es noch nicht trainiert ist*

In [57]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encoded_tensor.shape)

model.eval()
out = generate_text_simple(
model=model,
idx=encoded_tensor,
max_new_tokens=6,
context_size=GPT_CONFIG_124M["context_length"]
)
print("Output:", out)
print("Output length:", len(out[0]))

decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])
Output: tensor([[15496,    11,   314,   716, 21203, 47522, 12355, 29196, 20122, 45721]])
Output length: 10
Hello, I am VictoryFacjava directoriesheartedAPE


#### Sources:

Raschka, Sebastian (2025): Build a Large Language Model (from scratch). Shelter Island: Manning (From scratch series). Online verfügbar unter https://ebookcentral.proquest.com/lib/kxp/detail.action?docID=31657639.